In [1]:
addprocs(6);

In [2]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    import POMDPToolbox.PreviousObservation

    import POMCP.belief_from_node
    import POMCP.init_V
    # import POMCP.POMCPPolicy
    import POMDPs.action
end

In [3]:
N = 100;
eps = 0.01;
problem = BabyPOMDP(-5, -10);

In [4]:
@everywhere type RandomBabyPolicy <: POMDPs.Policy
    rng::AbstractRNG
end
@everywhere action(p::RandomBabyPolicy, b::POMDPs.Belief) = BabyAction(rand(p.rng)>0.5)

In [5]:
function est_reward(problem, policy, belief, N)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        POMDPs.simulate(problem, policy, belief, rng=sim_rng, eps=eps, initial_state=BabyState(false))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [6]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
est_reward(problem, FeedWhenCrying(), PreviousObservation(BabyObservation(false)), N)

-15.061634270536405

In [7]:
# Random
pol_rng = MersenneTwister(7)
est_reward(problem, RandomBabyPolicy(pol_rng), PreviousObservation(BabyObservation(false)), N)

-31.16839941321846

In [8]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(FeedWhenCrying(),
                     0.01,
                     10,
                     300, 
                     rng,
                     false,
                     PreviousObservationConverter())

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(BabyStateDistribution(0.0)), N)

elapsed time: 24.468402263 seconds (11037700 bytes allocated)


-14.64198108450179

In [9]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(RandomBabyPolicy(rollout_pol_rng),
                     0.01,
                     10,
                     300, 
                     rng,
                     false,
                     EmptyConverter())

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(BabyStateDistribution(0.0)), N)

elapsed time: 17.123901163 seconds (1122584 bytes allocated)


-16.490674822957597

In [10]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy
    end
    action(p::OptBabyPolicy, b::BabyStateDistribution) = BabyAction(b.p_hungry>0.28206)
end
est_reward(problem, OptBabyPolicy(), BabyStateDistribution(0.0), N)

-14.686851194997828